In [9]:
import pickle
from pathlib import Path
from transformers import ViTFeatureExtractor, ViTImageProcessor
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import requests

In [10]:
# Configuration
INPUT_PKL = Path("/Users/samtett/Documents/Python/Project/replay_buffer_random_orientation.pkl")
CLIP_OUTPUT = INPUT_PKL.parent / "replay_buffer_with_clip_embeddings.pkl"
VIT_OUTPUT = INPUT_PKL.parent / "replay_buffer_with_vit_embeddings.pkl"

In [11]:
class CLIPPatcher:
    def __init__(self, ckpt="openai/clip-vit-base-patch32", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.proc   = CLIPProcessor.from_pretrained(ckpt)
        self.model  = CLIPModel.from_pretrained(ckpt).to(self.device).eval()

    @staticmethod
    def _l2(x):
        return x / x.norm(dim=-1, keepdim=True).clamp(min=1e-6)

    @torch.no_grad()
    def get_joint_embeddings(self, image: Image.Image, prompts: list[str]) -> np.ndarray:
        if not prompts:
            h = self.model.config.projection_dim
            return np.empty((0, h), dtype=np.float32)

        img_inp = self.proc(images=image, return_tensors="pt").to(self.device)
        img_feat = self._l2(self.model.get_image_features(**img_inp))
        txt_inp = self.proc(text=prompts, return_tensors="pt", padding=True).to(self.device)
        txt_feat = self._l2(self.model.get_text_features(**txt_inp))
        joint = self._l2(img_feat + txt_feat)
        return joint.cpu().numpy().copy()

def compute_goal_embeddings(rgb_array, tasks_dict, goal_index, color_index, embedding_size):
    pass

In [12]:
def main():
    matcher = CLIPMatcher()
    matcher = CLIPMatcher()
    print("🚀 Loading replay buffer...")
    with INPUT_PKL.open("rb") as f:
        buffer = pickle.load(f)

    # test_limit = 1
    print(f"Buffer size: {len(buffer.buffer)}")
    
    observations = np.array([experience[0] for experience in buffer.buffer])#[:test_limi
    actions = np.array([experience[1] for experience in buffer.buffer])#[:test_limit]
    rewards = np.array([experience[2] for experience in buffer.buffer])#[:test_limit]
    # next_observations = np.array([experience[3] for experience in buffer.buffer])#[:test_limit]
    dones = np.array([experience[4] for experience in buffer.buffer])#[:test_limit]

    rgb = np.array([obs['rgb'] for obs in observations]).transpose(0, 1, 4, 2, 3)  # (episodes, steps, channels, height, width)
    # next_rgb = np.array([obs['rgb'] for obs in next_observations]).transpose(0, 1, 4, 2, 3)  # (episodes, steps, channels, height, width)
    goal_index = np.array([obs['goal_index'] for obs in observations])  # (episodes, steps, 1)

    if rgb is None or goal_index is None:
        print("❌ Missing required data in buffer: rgb, next_rgb or goal_index.")
        return

    num_episodes, num_steps = rgb.shape[0], rgb.shape[1]
    print(f"✅ Loaded buffer with {num_episodes} episodes, {num_steps} steps each.")

    print("💬 Computing joint embeddings...")
    for ep in tqdm(range(num_episodes), desc="Embedding Episodes"):
        for st in range(num_steps):
            task_list = tasks_dict[ep][st]

            img_np = rgb[ep, st]
            if isinstance(img_np, torch.Tensor):
                img_np = img_np.cpu().numpy()
            img_np = img_np.transpose(1, 2, 0).astype(np.uint8)
            if img_np.shape[-1] == 4:
                img_np = img_np[..., :3]

            img_pil = Image.fromarray(img_np)

            if not task_list:
                # No object in frame → Add "move around" task
                prompts = ["No balls in frame, move around"]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                tasks_dict[ep][st] = [{
                    "task": prompts[0],
                    "location": None,
                    "colour": None,
                    "bbox": None,
                    "embedding": embs[0]
                }]
            else:
                prompts = [t["task"] for t in task_list]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                for t, e in zip(task_list, embs):
                    t["embedding"] = e

            if not task_list:
                # No object in frame → Add "move around" task
                prompts = ["No balls in frame, move around"]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                tasks_dict[ep][st] = [{
                    "task": prompts[0],
                    "location": None,
                    "colour": None,
                    "bbox": None,
                    "embedding": embs[0]
                }]
            else:
                prompts = [t["task"] for t in task_list]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                for t, e in zip(task_list, embs):
                    t["embedding"] = e

    # Compute final embeddings based on goal_index
    print("📦 Computing goal-based embeddings...")
    embedding_size = matcher.model.config.text_config.projection_size
    print(f"Embedding size: {embedding_size}")
    color_index = {
        "red": 0,
        "green": 1,
        "blue": 2,
        "yellow": 3,
        "magenta": 4
    }

    embeddings = compute_goal_embeddings(rgb, tasks_dict, goal_index, color_index, embedding_size)

    buffer_with_embeddings = []
    buffer_with_embeddings.append(embeddings)
    buffer_with_embeddings.append(actions)
    buffer_with_embeddings.append(rewards)
    buffer_with_embeddings.append(dones)

In [18]:
# Save
print("💾 Saving final buffer with embeddings...")
with CLIP_OUTPUT.open("wb") as f:
        pickle.dump(CLIP_OUTPUT, f, protocol=pickle.HIGHEST_PROTOCOL)
        pickle.dump(CLIP_OUTPUT, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"✅ Done. Embeddings saved to: {CLIP_OUTPUT}")
print(f"✅ Done. Embeddings saved to: {CLIP_OUTPUT}")

if __name__ == "__main__":
    main()

💾 Saving final buffer with embeddings...
✅ Done. Embeddings saved to: /Users/samtett/Documents/Python/Project/replay_buffer_with_clip_embeddings.pkl
✅ Done. Embeddings saved to: /Users/samtett/Documents/Python/Project/replay_buffer_with_clip_embeddings.pkl


NameError: name 'CLIPMatcher' is not defined

In [ ]:
# Save a single object (e.g., a list of embeddings)
with open('clip_embeddings.pkl', 'wb') as f:
    pickle.dump(CLIP_OUTPUT, f, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# Load existing data
with open(INPUT_PKL, "rb") as f:
    buffer = pickle.load(f)

In [ ]:

from PIL import Image
import torch

# Load CLIP
clip_model_id = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_id)
clip_processor = CLIPProcessor.from_pretrained(clip_model_id)
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = clip_model.to(device)

# Image Embedding
def get_clip_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        img_emb = clip_model.get_image_features(**inputs)
    return img_emb.cpu().numpy().squeeze()

# Text Embedding
def get_clip_text_embedding(text):
    inputs = clip_processor(text=[text], return_tensors="pt").to(device)
    with torch.no_grad():
        text_emb = clip_model.get_text_features(**inputs)
    return text_emb.cpu().numpy().squeeze()

def compute_goal_embeddings(rgb_array, tasks_dict, goal_index, color_index, embedding_size):
    pass



In [37]:
from transformers import ViTFeatureExtractor, ViTModel

# Load ViT
vit_model_id = "google/vit-base-patch16-224"
vit_model = ViTModel.from_pretrained(vit_model_id)
vit_processor = ViTFeatureExtractor.from_pretrained(vit_model_id)
vit_model = vit_model.to(device)

# Image Embedding
def get_vit_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = vit_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = vit_model(**inputs)
        # Use the [CLS] token embedding as the image representation
        img_emb = outputs.last_hidden_state[:, 0, :]
    return img_emb.cpu().numpy().squeeze()

def compute_goal_embeddings(rgb_array, tasks_dict, goal_index, color_index, embedding_size):
    pass


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/samtett/Documents/Python/.venv/lib/python3.13/site-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [31]:
# Save a single object (e.g., a list of embeddings)
with open('clip_embeddings.pkl', 'wb') as f:
    pickle.dump(CLIP_OUTPUT, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('vit_embeddings.pkl', 'wb') as f:
    pickle.dump(VIT_OUTPUT, f, protocol=pickle.HIGHEST_PROTOCOL)
